In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


Define some variables<br>
The images are scaled to a size of 224x224 pixels. The reason for this is that the pre-trained model `MobileNetV3Small` is used. Its weights were trained for RGB images of size 224x224.

In [2]:
data_loader_path = "data/train/"
model_path = "model/"
image_height = 224
image_width = 224
train_epochs = 30
input_shape = (image_height,image_width,3)

For the training of the model, data generators are created. One for the training data and one for the validation data.<br>
The data can also be normalized via the `ImageDataGenerator`. However, this is not necessary for the `MobileNet` model.<br>
The pixel values have to be passed here in the range from 0 to 255.

In [3]:
train_datagen = ImageDataGenerator(validation_split=0.2)

train_it = train_datagen.flow_from_directory(data_loader_path, target_size=(image_height, image_width), color_mode='rgb', class_mode='sparse', batch_size=128, subset='training')
val_it = train_datagen.flow_from_directory(data_loader_path, target_size=(image_height, image_width), color_mode='rgb', class_mode='sparse', batch_size=128, subset='validation')

Found 16000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


As mentioned before, the `MobileNetV3Small` from the TensorFlow library is used. Here the `Imagenet` weights are used.<br>
The last layers which are important for the classification are not used by the pre-trained network. Instead, two own layers are added to the model.<br>
Furthermore, only the last 10 layers of the model are trained.

In [4]:
base_model_MobileNetV3Small = tf.keras.applications.MobileNetV3Small(weights= 'imagenet',
                                                                     input_shape=input_shape,
                                                                     include_top=False)    # False, do not include
                                                                                           # the classification 
                                                                                           # layer of the model

x = base_model_MobileNetV3Small.layers[-1].output
x = tf.keras.layers.GlobalAveragePooling2D()(x)

# Add own classififcation layer
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)
model_MobileNetV3Small = tf.keras.Model(base_model_MobileNetV3Small.input, outputs)

# Set only last 10 layers trainable
for layer in model_MobileNetV3Small.layers[:-10]:
    layer.trainable = False

model_MobileNetV3Small.summary()

model_MobileNetV3Small.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["acc"])

callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True), tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3)]

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv (Conv2D)                   (None, 112, 112, 16) 432         rescaling[0][0]                  
__________________________________________________________________________________________________
Conv/BatchNorm (BatchNormalizat (None, 112, 112, 16) 64          Conv[0][0]                       
______________________________________________________________________________________________

The training of the model can take a longer time.

In [5]:
model_MobileNetV3Small.fit(train_it, steps_per_epoch=len(train_it),validation_data=val_it, validation_steps=len(val_it), epochs=train_epochs, callbacks=callbacks)

Epoch 1/100
125/125 [==============================] - 102s 747ms/step - loss: 0.4323 - acc: 0.7899 - val_loss: 0.3690 - val_acc: 0.8345
Epoch 2/100
125/125 [==============================] - 92s 737ms/step - loss: 0.3547 - acc: 0.8397 - val_loss: 0.3578 - val_acc: 0.8330
Epoch 3/100
125/125 [==============================] - 92s 740ms/step - loss: 0.3367 - acc: 0.8520 - val_loss: 0.3601 - val_acc: 0.8438
Epoch 4/100
125/125 [==============================] - 97s 774ms/step - loss: 0.3227 - acc: 0.8571 - val_loss: 0.3470 - val_acc: 0.8422
Epoch 5/100
125/125 [==============================] - 91s 725ms/step - loss: 0.3108 - acc: 0.8637 - val_loss: 0.3556 - val_acc: 0.8305
Epoch 6/100
125/125 [==============================] - 87s 696ms/step - loss: 0.3016 - acc: 0.8686 - val_loss: 0.3450 - val_acc: 0.8415
Epoch 7/100
125/125 [==============================] - 87s 695ms/step - loss: 0.2928 - acc: 0.8736 - val_loss: 0.3406 - val_acc: 0.8447
Epoch 8/100
125/125 [==========================

In [6]:
if not os.path.isdir(model_path):
    os.mkdir(model_path)
model_MobileNetV3Small.save(model_path + "person_detection_MobileNetV3Small.h5")

C:\Users\dk100\AppData\Roaming\Python\Python38\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
